## **Part - 3 Review Summarization Using Generative AI**
Objective: Summarize reviews into articles that recommend the top products for each category.

Task: Create a model that generates a short article (like a blog post) for each product category. The output should include:

- Top 3 products and key differences between them.
- Top complaints for each of those products.
- Worst product in the category and why it should be avoided.

#### Imports.

In [14]:
import os
import re
import json
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, BitsAndBytesConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# load HugginFace token to environment
if not os.environ.get('HF_TOKEN'):
    os.environ['HF_TOKEN'] = input('Enter API token for Hugging Face: ')
else:
    print('Hugging Face token already loaded to environment')

#### Load the Dataset
We merge the datasets

In [8]:
# First, load both dataframes again for clarity
df_reviews = pd.read_csv(r'.\Dataset\cleaned_reviews.csv')
df_clusters = pd.read_csv(r'.\categories_cluster.csv')

# Normalize category strings to ensure better matching
df_reviews['categories'] = df_reviews['categories'].str.lower()
df_clusters['Category'] = df_clusters['Category'].str.lower()

# Explode reviews that have multiple categories
df_reviews['categories'] = df_reviews['categories'].str.split(',')
df_exploded = df_reviews.explode('categories')
df_exploded['categories'] = df_exploded['categories'].str.strip()

# Merge on the individual category
merged_df = df_exploded.merge(df_clusters, how='left', left_on='categories', right_on='Category')

# Show the first few rows of the merged dataframe (with cluster categories)
merged_df[['name', 'brand', 'categories', 'reviews.rating', 'reviews', 'cluster_categories']].head(10)

,name,brand,categories,reviews.rating,reviews,cluster_categories
0,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,featured brands,5.0,Great device for reading. Definately pricey.. ...,General Electronics & Household
1,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,e-readers & accessories,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
2,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,ebook readers,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
3,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,tablets,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
4,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,amazon book reader,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
5,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,electronics,5.0,Great device for reading. Definately pricey.. ...,General Electronics & Household
6,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,computers & tablets,5.0,Great device for reading. Definately pricey.. ...,NaN
7,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,amazon ereaders,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
8,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,kindle e-readers,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets
9,All-New Kindle Oasis E-reader - 7 High-Resolut...,Amazon,amazon devices,5.0,Great device for reading. Definately pricey.. ...,Ebook readers & Tablets


##### **Prompt Fine-Tuning using T5-Large with 4-bit quantization**
fast inference with reduced memory usage. Getting strong generation capabilities without the heavy compute load.

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model_name = "t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def run_inference_on_model(prompt: str, max_tokens: int = 150) -> str:
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Now try it
output = run_inference_on_model(prompt)
print(output)


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend